In [1]:
import pandas as pd
import numpy as np
import time
import datetime
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import RFE
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf

In [2]:
weather_data = pd.read_csv("D:\ML\data\weatherAUS.csv", delimiter=',')
weather_data

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.4,0.0,NaN,NaN,E,31.0,SE,...,51.0,24.0,1024.6,1020.3,NaN,NaN,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.3,0.0,NaN,NaN,NNW,22.0,SE,...,56.0,21.0,1023.5,1019.1,NaN,NaN,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.9,0.0,NaN,NaN,N,37.0,SE,...,53.0,24.0,1021.0,1016.8,NaN,NaN,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.0,0.0,NaN,NaN,SE,28.0,SSE,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [3]:
weather_data['Date'] = pd.to_datetime(weather_data['Date'])

weather_data['Day'] = weather_data['Date'].dt.day
weather_data['Month'] = weather_data['Date'].dt.month
weather_data['Year'] = weather_data['Date'].dt.year

weather_data = weather_data.drop(['Date'], axis=1)

weather_data['RainTomorrow'] = weather_data['RainTomorrow'].replace(['Yes'], '1')
weather_data['RainTomorrow'] = weather_data['RainTomorrow'].replace(['No'], '0')
weather_data

categorical_cols = ['WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']
numeric_cols = list(set(weather_data.columns.values.tolist()) - set(categorical_cols))

weather_data[numeric_cols] = weather_data[numeric_cols].fillna(weather_data[numeric_cols].mean())

weather_data[categorical_cols] = np.where(weather_data[categorical_cols].isnull(), weather_data[categorical_cols].mode(), weather_data[categorical_cols])

weather_data = pd.get_dummies(weather_data, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'])

weather_data = weather_data.astype(float)

weather_data.to_csv(r'D:\ML\data\weather_data.csv', index= False)
weather_data

C:\Users\radio\AppData\Local\Temp\ipykernel_9512\4198551779.py:16: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  weather_data[numeric_cols] = weather_data[numeric_cols].fillna(weather_data[numeric_cols].mean())


,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,...,WindDir3pm_S,WindDir3pm_SE,WindDir3pm_SSE,WindDir3pm_SSW,WindDir3pm_SW,WindDir3pm_W,WindDir3pm_WNW,WindDir3pm_WSW,RainToday_No,RainToday_Yes
0,13.4,22.900000,0.6,5.468232,7.611178,44.00000,20.0,24.0,71.0,22.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,7.4,25.100000,0.0,5.468232,7.611178,44.00000,4.0,22.0,44.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,12.9,25.700000,0.0,5.468232,7.611178,46.00000,19.0,26.0,38.0,30.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,9.2,28.000000,0.0,5.468232,7.611178,24.00000,11.0,9.0,45.0,16.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,17.5,32.300000,1.0,5.468232,7.611178,41.00000,7.0,20.0,82.0,33.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2.8,23.400000,0.0,5.468232,7.611178,31.00000,13.0,11.0,51.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
145456,3.6,25.300000,0.0,5.468232,7.611178,22.00000,13.0,9.0,56.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
145457,5.4,26.900000,0.0,5.468232,7.611178,37.00000,9.0,9.0,53.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
145458,7.8,27.000000,0.0,5.468232,7.611178,28.00000,13.0,7.0,51.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(weather_data)

weather_data = scaler.transform(weather_data)
weather_data = pd.DataFrame(weather_data)

weather_data = weather_data[:10000]

weather_data

,0,1,2,3,4,5,6,7,8,9,...,109,110,111,112,113,114,115,116,117,118
0,0.516509,0.523629,0.001617,0.037712,0.524909,0.294574,0.153846,0.275862,0.71,0.22,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,0.375000,0.565217,0.000000,0.037712,0.524909,0.294574,0.030769,0.252874,0.44,0.25,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
2,0.504717,0.576560,0.000000,0.037712,0.524909,0.310078,0.146154,0.298851,0.38,0.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.417453,0.620038,0.000000,0.037712,0.524909,0.139535,0.084615,0.103448,0.45,0.16,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.613208,0.701323,0.002695,0.037712,0.524909,0.271318,0.053846,0.229885,0.82,0.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.417453,0.500945,0.000000,0.020690,0.744828,0.085271,0.030769,0.126437,0.71,0.51,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9996,0.483491,0.419660,0.000539,0.023448,0.048276,0.186047,0.100000,0.229885,0.83,0.96,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9997,0.488208,0.485822,0.067385,0.004138,0.737931,0.224806,0.130769,0.172414,0.70,0.62,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
9998,0.462264,0.519849,0.000000,0.016552,0.662069,0.286822,0.046154,0.103448,0.54,0.79,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [5]:
balance = len(weather_data.loc[weather_data[16]==0])//len(weather_data.loc[weather_data[16]==1])

data_sample = weather_data.loc[weather_data[16]==1]
data_sample = data_sample.loc[data_sample.index.repeat(balance)]
data_n = pd.concat([weather_data.loc[weather_data[16]==0], data_sample]).sample(frac=1)

print(data_n[16].value_counts())

0.0    8101
1.0    7596
Name: 16, dtype: int64


In [6]:
y_class = data_n[16]
X_class = data_n.drop([16], axis=1)

In [7]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier().fit(X_class, y_class)
rfe = RFE(estimator=tree, n_features_to_select=15, step=2).fit(X_class, y_class)
X_rfe = pd.DataFrame(rfe.transform(X_class), columns=rfe.get_feature_names_out())
X_rfe

,x0,x1,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17
0,0.443396,0.614367,0.193798,0.000000,0.149425,0.760000,0.340000,0.636364,0.6368,0.494162,0.501103,0.466245,0.627639,0.800000,0.181818
1,0.582547,0.459357,0.426357,0.215385,0.402299,0.930000,0.920000,0.672727,0.6560,0.888889,0.888889,0.535865,0.466411,0.100000,0.818182
2,0.698113,0.665406,0.325581,0.153846,0.356322,0.680000,0.800000,0.520661,0.4896,0.333333,0.777778,0.710970,0.614203,0.100000,0.181818
3,0.417453,0.425331,0.294574,0.053846,0.229885,0.800000,0.450000,0.606612,0.6128,0.777778,0.777778,0.413502,0.395393,0.000000,0.909091
4,0.589623,0.476371,0.240310,0.115385,0.149425,0.990000,0.930000,0.593388,0.6176,0.494162,0.501103,0.535865,0.487524,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15692,0.521226,0.489603,0.100775,0.069231,0.080460,0.770000,0.600000,0.685950,0.6608,0.494162,0.501103,0.523207,0.493282,0.733333,0.272727
15693,0.608491,0.635161,0.186047,0.100000,0.149425,0.420000,0.270000,0.593388,0.5904,0.777778,0.777778,0.611814,0.604607,0.766667,1.000000
15694,0.625000,0.765595,0.255814,0.015385,0.218391,0.610000,0.400000,0.507438,0.4960,0.494162,0.501103,0.738397,0.752399,0.266667,0.000000
15695,0.792453,0.869565,0.325581,0.146154,0.229885,0.380000,0.120000,0.447934,0.4384,0.333333,0.777778,0.765823,0.865643,0.400000,0.090909


In [8]:
def test_class(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)
    bag = BaggingClassifier().fit(X_train, y_train)
    print(classification_report(y_test, bag.predict(X_test)))

In [9]:
test_class(X_rfe, y_class)

              precision    recall  f1-score   support

         0.0       0.98      0.91      0.95      2436
         1.0       0.91      0.98      0.95      2274

    accuracy                           0.95      4710
   macro avg       0.95      0.95      0.95      4710
weighted avg       0.95      0.95      0.95      4710



In [10]:
X_rfe.rename(columns = {'x0':'MinTemp', 'x1':'MaxTemp', 'x2':'Rainfall', 'x5':'WindGustSpeed', 'x6':'WindSpeed9am', 'x7':'WindSpeed3pm', 
                        'x8':'Humidity9am', 'x9':'Humidity3pm', 'x10':'Pressure9am', 'x11':'Pressure3pm', 'x12':'Cloud9am', 
                        'x13':'Cloud3pm', 'x14':'Temp9am', 'x15':'Temp3pm', 'x16':'RainTomorrow', 'x17':'Day'}, inplace = True )

In [11]:
X_rfe.to_csv(r'D:\ML\data\weather_data1.csv', index= False)
X_rfe

,MinTemp,MaxTemp,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,Day
0,0.443396,0.614367,0.193798,0.000000,0.149425,0.760000,0.340000,0.636364,0.6368,0.494162,0.501103,0.466245,0.627639,0.800000,0.181818
1,0.582547,0.459357,0.426357,0.215385,0.402299,0.930000,0.920000,0.672727,0.6560,0.888889,0.888889,0.535865,0.466411,0.100000,0.818182
2,0.698113,0.665406,0.325581,0.153846,0.356322,0.680000,0.800000,0.520661,0.4896,0.333333,0.777778,0.710970,0.614203,0.100000,0.181818
3,0.417453,0.425331,0.294574,0.053846,0.229885,0.800000,0.450000,0.606612,0.6128,0.777778,0.777778,0.413502,0.395393,0.000000,0.909091
4,0.589623,0.476371,0.240310,0.115385,0.149425,0.990000,0.930000,0.593388,0.6176,0.494162,0.501103,0.535865,0.487524,0.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15692,0.521226,0.489603,0.100775,0.069231,0.080460,0.770000,0.600000,0.685950,0.6608,0.494162,0.501103,0.523207,0.493282,0.733333,0.272727
15693,0.608491,0.635161,0.186047,0.100000,0.149425,0.420000,0.270000,0.593388,0.5904,0.777778,0.777778,0.611814,0.604607,0.766667,1.000000
15694,0.625000,0.765595,0.255814,0.015385,0.218391,0.610000,0.400000,0.507438,0.4960,0.494162,0.501103,0.738397,0.752399,0.266667,0.000000
15695,0.792453,0.869565,0.325581,0.146154,0.229885,0.380000,0.120000,0.447934,0.4384,0.333333,0.777778,0.765823,0.865643,0.400000,0.090909


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X_rfe, y_class, test_size=0.33, random_state=42)

In [13]:
print(X_test.iloc[1270])

MinTemp          0.485849
MaxTemp          0.516068
Rainfall         0.000000
WindGustSpeed    0.372093
WindSpeed9am     0.169231
WindSpeed3pm     0.252874
Humidity9am      0.430000
Humidity3pm      0.410000
Pressure9am      0.578512
Pressure3pm      0.536000
Cloud3pm         0.888889
Temp9am          0.489451
Temp3pm          0.483685
RainTomorrow     0.600000
Day              0.636364
Name: 2069, dtype: float64


In [14]:
print(y_test.iloc[1270])

1.0


In [15]:
knn = KNeighborsClassifier(n_neighbors=3, metric='minkowski', p=2)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [16]:
y_bin_knn_pred = knn.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_bin_knn_pred)

0.8562053657595059

In [18]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.83      0.81      2704
         1.0       0.80      0.77      0.79      2477

    accuracy                           0.80      5181
   macro avg       0.80      0.80      0.80      5181
weighted avg       0.80      0.80      0.80      5181



In [19]:
from sklearn import svm

clf1 = svm.SVC()
clf1.fit(X_train, y_train)

y_pred1 = clf1.predict(X_test)
print(classification_report(y_test, y_pred1))

              precision    recall  f1-score   support

         0.0       0.81      0.85      0.83      2704
         1.0       0.83      0.78      0.80      2477

    accuracy                           0.82      5181
   macro avg       0.82      0.81      0.82      5181
weighted avg       0.82      0.82      0.82      5181



In [20]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.74      0.87      0.80      2704
         1.0       0.82      0.66      0.73      2477

    accuracy                           0.77      5181
   macro avg       0.78      0.76      0.76      5181
weighted avg       0.78      0.77      0.77      5181



In [21]:
#knn справился лучше всех

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier

bagclf = BaggingClassifier(base_estimator=RandomForestClassifier(n_estimators = 100), n_estimators=200, random_state=0)
bagclf.fit(X_train, y_train)

D:\ML\venv\lib\site-packages\sklearn\ensemble\_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=RandomForestClassifier(), n_estimators=200,
                  random_state=0)

In [23]:
y_pred_bag = bagclf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.74      0.87      0.80      2704
         1.0       0.82      0.66      0.73      2477

    accuracy                           0.77      5181
   macro avg       0.78      0.76      0.76      5181
weighted avg       0.78      0.77      0.77      5181



In [24]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(n_estimators=200, learning_rate=0.6, max_depth=3, random_state=0)
gb_clf.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=0.6, n_estimators=200, random_state=0)

In [25]:
y_pred_gb = gb_clf.predict(X_test)
print(classification_report(y_test, y_pred_gb))

              precision    recall  f1-score   support

         0.0       0.94      0.87      0.91      2704
         1.0       0.87      0.94      0.91      2477

    accuracy                           0.91      5181
   macro avg       0.91      0.91      0.91      5181
weighted avg       0.91      0.91      0.91      5181



In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline

estimators = [
            ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
            ('svr', make_pipeline(StandardScaler(), LinearSVC(random_state=42)))]

clf_stc = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf_stc.fit(X_train, y_train)

D:\ML\venv\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
D:\ML\venv\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
D:\ML\venv\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
D:\ML\venv\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
D:\ML\venv\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
D:\ML\venv\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


StackingClassifier(estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('svr',
                                Pipeline(steps=[('standardscaler',
                                                 StandardScaler()),
                                                ('linearsvc',
                                                 LinearSVC(random_state=42))]))],
                   final_estimator=LogisticRegression())

In [27]:
y_pred_stc = clf_stc.predict(X_test)
print(classification_report(y_test, y_pred_stc))

              precision    recall  f1-score   support

         0.0       0.98      0.91      0.95      2704
         1.0       0.91      0.98      0.94      2477

    accuracy                           0.94      5181
   macro avg       0.95      0.95      0.94      5181
weighted avg       0.95      0.94      0.95      5181



In [28]:
import pickle
# save the model to disk
filename = 'ModelKnn.sav'
pickle.dump(knn, open(filename, 'wb'))

filename = 'ModelStacking.sav'
pickle.dump(clf_stc, open(filename, 'wb'))

# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))

#knn.save('../models/ModelKnn')
#clf_stc.save('../models/ModelStacking')

In [13]:
w0 = 1 / y_train[y_train==0].shape[0]
w1 = 1 / y_train[y_train==1].shape[0]

In [24]:
import tensorflow as tf

model_classification = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(15,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model_classification.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="binary_crossentropy")
model_classification.fit(X_train, y_train, epochs=50, verbose=None,
                           class_weight={0: w0, 1: w1})
y_pred = np.around(model_classification.predict(X_test, verbose=None))

In [25]:
y_pred = [np.argmax(pred) for pred in model_classification.predict(X_test, verbose=None)]

In [26]:
#print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.52      1.00      0.69      2699
         1.0       0.00      0.00      0.00      2482

    accuracy                           0.52      5181
   macro avg       0.26      0.50      0.34      5181
weighted avg       0.27      0.52      0.36      5181



D:\ML\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ML\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\ML\venv\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
model_json = model_classification.to_json() 
with open("model_classification.json", "w") as json_file:
    json_file.write(model_json)
model_classification.save_weights("model_weights.h5")
print("Saved model to disk")

Saved model to disk
